In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("lab4").getOrCreate()
from pyspark.sql.functions import *
from pyspark.sql.types import *

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/03 16:02:54 WARN Utils: Your hostname, dbl-12, resolves to a loopback address: 127.0.1.1; using 172.16.58.146 instead (on interface eno1)
25/09/03 16:02:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/03 16:02:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.json('movies.json')


[Stage 0:>                                                        (0 + 14) / 14]



In [3]:
df.show()

+-----------+----------+--------------------+--------------------+-----+--------------------+----------+--------------+
|helpfulness|product_id|        profile_name|              review|score|             summary|      time|       user_id|
+-----------+----------+--------------------+--------------------+-----+--------------------+----------+--------------+
|        7/7|B003AI2VGA|Brian E. Erland "...|Synopsis: On the ...|  3.0|"There Is So Much...|1182729600|A141HP4LYPWMSR|
|        4/4|B003AI2VGA|          Grady Harp|THE VIRGIN OF JUA...|  3.0|Worthwhile and Im...|1181952000|A328S9RN3U5M68|
|       8/10|B003AI2VGA|Chrissy K. McVay ...|The scenes in thi...|  5.0|This movie needed...|1164844800|A1I7QGUDP043DG|
|        1/1|B003AI2VGA|        golgotha.gov|THE VIRGIN OF JUA...|  3.0|distantly based o...|1197158400|A1M5405JH9THP9|
|        1/1|B003AI2VGA|KerrLines "&#34;M...|Informationally, ...|  3.0|"What's going on ...|1188345600| ATXL536YX71TR|
|        0/0|B003AI2VGA|abra "a devoted 

In [4]:
def bin_from_hex(hash_string):
    return int(hash_string, 16)%10**8

In [5]:
bin_from_hex_udf = udf(bin_from_hex, LongType())

In [6]:
df = df.withColumn('user_id_preprocessed', bin_from_hex_udf(sha1(col('user_id'))))
df = df.withColumn('product_id_preprocessed', bin_from_hex_udf(sha1(col('product_id'))))

In [7]:
df.show()

+-----------+----------+--------------------+--------------------+-----+--------------------+----------+--------------+--------------------+-----------------------+
|helpfulness|product_id|        profile_name|              review|score|             summary|      time|       user_id|user_id_preprocessed|product_id_preprocessed|
+-----------+----------+--------------------+--------------------+-----+--------------------+----------+--------------+--------------------+-----------------------+
|        7/7|B003AI2VGA|Brian E. Erland "...|Synopsis: On the ...|  3.0|"There Is So Much...|1182729600|A141HP4LYPWMSR|             5460385|               51259877|
|        4/4|B003AI2VGA|          Grady Harp|THE VIRGIN OF JUA...|  3.0|Worthwhile and Im...|1181952000|A328S9RN3U5M68|            64843361|               51259877|
|       8/10|B003AI2VGA|Chrissy K. McVay ...|The scenes in thi...|  5.0|This movie needed...|1164844800|A1I7QGUDP043DG|             1480848|               51259877|
|        1

In [8]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

train, test = df.randomSplit([0.6, 0.4])

In [9]:
train.count()

30133

In [10]:
test.count()

19867

In [11]:
als = ALS(userCol='user_id_preprocessed', itemCol='product_id_preprocessed', ratingCol='score', coldStartStrategy='drop')
model = als.fit(train)
predictions = model.transform(test)
print(predictions.filter(col('prediction').isNaN()).count())
predictions = predictions.dropna()

0


In [12]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="score", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
rmse

2.268563536578244

In [14]:
from pyspark.ml.tuning import ParamGridBuilder

In [16]:
grid = ParamGridBuilder.addGrid(param=)

TypeError: ParamGridBuilder.addGrid() missing 3 required positional arguments: 'self', 'param', and 'values'